# This code reads CloudSat Data and derive the grid-level Probability of Warm Rain

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
#from mpl_toolkits.basemap import Basemap, cm # To plot data on a map
from scipy import stats
import os,datetime,sys
from pyhdf.SD import SD, SDC
import pyhdf.HDF as HDF
import pyhdf.VS as VS
import glob
import pickle
from Cartopy_Lib import Cartopy_Global_Contour 
from CloudSat_Lib import *
import Utility_Lib as UL

In [3]:
UL.Nd_from_tau_re(np.array([10,15,20]),np.array([10,15,20]))

array([117.69666765,  52.30963007,  29.42416691])

In [4]:
class Get_file_lists():
    def __init__(self,start_date,end_date,
                GEOPROF_LIDAR_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_2B_GEOPROF_LIDAR/',
                 GEOPROF_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_2B_GEOPROF/',
                 MOD06_AUX_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_MOD06_AUX/',
                 ECMWF_AUX_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_ECMWF_AUX/',
                 savelist = False):
        
        self.GEOPROF_LIDAR_file_list = []
        self.GEOPROF_file_list = []
        self.MOD06_AUX_file_list = []
        self.ECMWF_AUX_file_list = []
        
        self.GEOPROF_LIDAR_dp_list = []
        self.GEOPROF_dp_list = []
        self.MOD06_AUX_dp_list = []
        self.ECMWF_AUX_dp_list = []
        
        d = start_date
        while d <= end_date:
            y = d.year
            subdir = '{:04d}/'.format(y)
            jday = (d - datetime.date(y,1,1)).days+1
            date_char = '{0:04d}{1:03d}'.format(y,jday)
            GEOPROF_LIDAR_flist =  glob.glob(GEOPROF_LIDAR_PATH+subdir+date_char+'*.hdf')
            if len(GEOPROF_LIDAR_flist) >0: 
                for GEOPROF_LIDAR_file in GEOPROF_LIDAR_flist:
                    p = GEOPROF_LIDAR_file.find(subdir)
                    time_stamp = GEOPROF_LIDAR_file[p+5:p+5+13]
                    MOD06_AUX_file =  glob.glob(MOD06_AUX_PATH+subdir+time_stamp+'*.hdf')
                    ECMWF_AUX_file =  glob.glob(ECMWF_AUX_PATH+subdir+time_stamp+'*.hdf')
                    GEOPROF_file   =  glob.glob(GEOPROF_PATH+subdir+time_stamp+'*.hdf')
                    if  len( MOD06_AUX_file) ==1 and len(ECMWF_AUX_file ) ==1 and len( GEOPROF_file ) ==1:
                        p = GEOPROF_LIDAR_file.find(subdir)
                        self.GEOPROF_LIDAR_file_list.append(GEOPROF_LIDAR_file[p+5:])
                        self.GEOPROF_LIDAR_dp_list.append(GEOPROF_LIDAR_file[0:p+5])
                        
                        p = GEOPROF_file[0].find(subdir)
                        self.GEOPROF_file_list.append(GEOPROF_file[0][p+5:])
                        self.GEOPROF_dp_list.append(GEOPROF_file[0][0:p+5])
                        
                        
                        #print(GEOPROF_LIDAR_file[p+5:])
                        p = MOD06_AUX_file[0].find(subdir)
                        self.MOD06_AUX_file_list.append(MOD06_AUX_file[0][p+5:])
                        self.MOD06_AUX_dp_list.append(MOD06_AUX_file[0][0:p+5])
                        
                        #print(MOD06_AUX_file[0][p+5:])
                        p = ECMWF_AUX_file[0].find(subdir)
                        self.ECMWF_AUX_file_list.append(ECMWF_AUX_file[0][p+5:])
                        self.ECMWF_AUX_dp_list.append(ECMWF_AUX_file[0][0:p+5])
                        #print(ECMWF_AUX_file[0][p+5:])
                        
                        
            d+=datetime.timedelta(1)
        if savelist:
            import pickle 
            filename = 'CF_filelist_'+start_date.strftime("%y%m%d") + '-'+end_date.strftime("%y%m%d")+'.pkl'
            pkl_savefile = open(filename, 'wb')
            pickle.dump(self, pkl_savefile, pickle.HIGHEST_PROTOCOL)
            pkl_savefile.close()
            

In [6]:
cdnc_hist_bound=np.logspace(0,np.log10(5000.0),31)
print(cdnc_hist_bound)

[1.00000000e+00 1.32830865e+00 1.76440386e+00 2.34367291e+00
 3.11312100e+00 4.13518554e+00 5.49280272e+00 7.29613735e+00
 9.69152234e+00 1.28733329e+01 1.70997595e+01 2.27137584e+01
 3.01708817e+01 4.00762431e+01 5.32336202e+01 7.07106781e+01
 9.39256053e+01 1.24762194e+02 1.65722701e+02 2.20130897e+02
 2.92401774e+02 3.88399805e+02 5.15914820e+02 6.85294117e+02
 9.10282102e+02 1.20913559e+03 1.60610526e+03 2.13340350e+03
 2.83381832e+03 3.76418539e+03 5.00000000e+03]


In [9]:
class CloudSat_Level3():
    def __init__(self,start_date,end_date,
                 cwp_hist_bound=np.logspace(0,np.log10(4000.0),21),
                 cer_hist_bound=np.linspace(0,30,20),
                 cdnc_hist_bound=np.logspace(0,np.log10(5000.0),31),
                 lts_hist_bound=np.linspace(0,30,22),
                 verbose=False,
                 savefile=False,
                 dbz_threshold=-15.0,
                 savepath='./'):
        GEOPROF_LIDAR_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_2B_GEOPROF_LIDAR/'
        GEOPROF_PATH = '/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_2B_GEOPROF/'
        MOD06_AUX_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_MOD06_AUX/'
        ECMWF_AUX_PATH='/umbc/rs/zzbatmos/common/Data/CloudSat/CloudSat_ECMWF_AUX/'

        #---- boundaries and grid points of Level 3 latitude and longitude-----#
        self.lat_size = 2.0
        self.lon_size = 4.0
        self.lat_bound = np.arange(-90,90+self.lat_size/2,self.lat_size)
        self.lon_bound = np.arange(-180,180+self.lon_size/2,self.lon_size)
        self.lat_grid = 0.5*(self.lat_bound[0:-1]+self.lat_bound[1:])
        self.lon_grid = 0.5*(self.lon_bound[0:-1]+self.lon_bound[1:])
        self.nlat = self.lat_grid.size
        self.nlon = self.lon_grid.size
        
        self.ncwp = len(cwp_hist_bound)
        self.ncer = len(cer_hist_bound)
        self.nlts = len(lts_hist_bound)
        self.ncdnc= len(cdnc_hist_bound)
        
        self.cwp_hist_bound = cwp_hist_bound
        self.cer_hist_bound = cer_hist_bound
        self.cdnc_hist_bound = cdnc_hist_bound
        self.lts_hist_bound = lts_hist_bound
        
        #print(self.lat_grid.shape,self.lon_grid.shape)
        
        self.total_obs = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.total_cloud = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.water_cloud = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.low_cloud = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.low_wcf = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.low_wcf_precip = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        
        self.SL_low_wcf = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.SL_low_wcf_precip = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        
        self.MODIS_tcf = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.MODIS_tcf_by_ctp = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.MODIS_wcf = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.MODIS_icf = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.MODIS_ucf = np.zeros([self.nlat,self.nlon],dtype=int).ravel()
        self.lwc_cwp_hist = np.zeros([self.nlat*self.nlon,self.ncwp])
        self.lwc_cer_hist = np.zeros([self.nlat*self.nlon,self.ncer])
        self.lwc_cdnc_hist = np.zeros([self.nlat*self.nlon,self.ncdnc])
        self.lwc_cwp_cer_jhist = np.zeros([self.nlat*self.nlon,self.ncwp,self.ncer])
        self.lwc_cwp_cdnc_jhist = np.zeros([self.nlat*self.nlon,self.ncwp,self.ncdnc])

        self.precip_lwc_cwp_hist = np.zeros([self.nlat*self.nlon,self.ncwp])
        self.precip_lwc_cer_hist = np.zeros([self.nlat*self.nlon,self.ncer])
        self.precip_lwc_cdnc_hist = np.zeros([self.nlat*self.nlon,self.ncdnc])
        self.precip_lwc_cwp_cer_jhist = np.zeros([self.nlat*self.nlon,self.ncwp,self.ncer])
        self.precip_lwc_cwp_cdnc_jhist = np.zeros([self.nlat*self.nlon,self.ncwp,self.ncdnc])

        
        #---- 
        #------ define MODIS properties
        MODIS_water_phase_index = 2
        MODIS_ice_phase_index = 3

        import time
        t0 = time.perf_counter()
        flist = Get_file_lists(start_date,end_date)
        nf = len(flist.GEOPROF_LIDAR_file_list)
        if verbose: print('total number of files:', nf)
        t1 = time.perf_counter()
        if verbose: print('finish compiling CloudSat file lists: using time [s]',t1-t0)
        for i in range(nf):
            if verbose: 
                t1 = time.perf_counter()
            #------- read in needed cloudSat files----------------#
            CS_2B_GEOPROF_LIDAR = CloudSat_2B_GEOPROF_LIDAR(flist.GEOPROF_LIDAR_dp_list[i],flist.GEOPROF_LIDAR_file_list[i])
            CS_2B_GEOPROF = CloudSat_2B_GEOPROF(flist.GEOPROF_dp_list[i],flist.GEOPROF_file_list[i])
            CS_MOD06_AUX = CloudSat_MOD06_AUX(flist.MOD06_AUX_dp_list[i],flist.MOD06_AUX_file_list[i])
            CS_ECMWF_AUX= CloudSat_ECMWF_AUX(flist.ECMWF_AUX_dp_list[i],flist.ECMWF_AUX_file_list[i])
            #------- sometimes the file dimensions are different, then go to next file 
            if len(CS_MOD06_AUX.lat) != len(CS_2B_GEOPROF_LIDAR.lat):
                if verbose: 
                    print('CS_MOD06_AUX and CS_2B_GEOPROF_LIDAR have the different dimension, ignore this file')
                continue
                    
            #------- put the lat and lon into the grids
            lat_index = np.searchsorted(self.lat_bound,CS_2B_GEOPROF_LIDAR.lat)
            lon_index = np.searchsorted(self.lon_bound,CS_2B_GEOPROF_LIDAR.lon)
            grid_index = lat_index * self.nlat + lon_index 
            grids = np.unique(grid_index)
            for ig in grids:
                current_grid_index = (grid_index==ig)
                
                #------ cloud fraction processing -------#
                self.total_obs[ig] += np.sum(current_grid_index)
                self.total_cloud[ig] += np.nansum(CS_2B_GEOPROF_LIDAR.total_cf[current_grid_index ])
                self.water_cloud[ig] += np.nansum((CS_MOD06_AUX.cldphase[current_grid_index ]== MODIS_water_phase_index))
                self.low_cloud[ig] += np.nansum((CS_2B_GEOPROF_LIDAR.low_cloud[current_grid_index ]))
                self.low_wcf[ig] += np.nansum((CS_2B_GEOPROF_LIDAR.low_cloud[current_grid_index ]) &\
                                              (CS_MOD06_AUX.cldphase[current_grid_index ]== MODIS_water_phase_index))
                self.SL_low_wcf[ig] += np.nansum((CS_2B_GEOPROF_LIDAR.single_layer_cloud[current_grid_index ]) &\
                                              (CS_2B_GEOPROF_LIDAR.low_cloud[current_grid_index ]) &\
                                              (CS_MOD06_AUX.cldphase[current_grid_index ]== MODIS_water_phase_index))
                
                self.low_wcf_precip[ig] += np.nansum((CS_2B_GEOPROF_LIDAR.low_cloud[current_grid_index]) &\
                                              (CS_MOD06_AUX.cldphase[current_grid_index]==MODIS_water_phase_index)&
                                              (CS_2B_GEOPROF.refl_max[current_grid_index]>dbz_threshold))
                self.SL_low_wcf_precip[ig] += np.nansum((CS_2B_GEOPROF_LIDAR.single_layer_cloud[current_grid_index ]) &\
                                              (CS_2B_GEOPROF_LIDAR.low_cloud[current_grid_index ]) &\
                                              (CS_MOD06_AUX.cldphase[current_grid_index ]== MODIS_water_phase_index)&
                                              (CS_2B_GEOPROF.refl_max[current_grid_index]>dbz_threshold))
                self.MODIS_tcf[ig] += np.nansum(CS_MOD06_AUX.cm1km[current_grid_index]<=1)
                self.MODIS_tcf_by_ctp[ig] += np.nansum(CS_MOD06_AUX.ctp[current_grid_index]>0.0)
                self.MODIS_wcf[ig] += np.nansum(CS_MOD06_AUX.cldphase[current_grid_index]==MODIS_water_phase_index)
                self.MODIS_icf[ig] += np.nansum(CS_MOD06_AUX.cldphase[current_grid_index]==MODIS_ice_phase_index)
                self.MODIS_ucf[ig] += np.nansum(CS_MOD06_AUX.cldphase[current_grid_index]==4)
                
                #------- cloud property processing -------#
                low_cloud_condition = (CS_2B_GEOPROF_LIDAR.low_cloud[current_grid_index])
                water_cloud_condition = (CS_MOD06_AUX.cldphase[current_grid_index]==MODIS_water_phase_index)
                precip_cloud_condition = (CS_2B_GEOPROF.refl_max[current_grid_index]>dbz_threshold)
                good_cwp_retrieval = (~np.isnan(CS_MOD06_AUX.cwp[current_grid_index]))
                good_cer_retrieval = (~np.isnan(CS_MOD06_AUX.cer[current_grid_index]))
                
                low_water_cloud_selection = low_cloud_condition & \
                                            water_cloud_condition & \
                                            good_cwp_retrieval & \
                                            good_cer_retrieval
                                    
                
                precip_low_water_cloud_selection = low_water_cloud_selection & precip_cloud_condition
                
                #------------ first process low water cloud ---------------------------#
                #------------ derive cwp histogram ---------------#
                cwp_subset = CS_MOD06_AUX.cwp[current_grid_index][low_water_cloud_selection]
                cwp_hist_index = np.searchsorted(self.cwp_hist_bound,cwp_subset)
                for icwp in np.unique(cwp_hist_index):
                    self.lwc_cwp_hist[ig,icwp] += np.sum(cwp_hist_index==icwp)
                #------------ derive cer histogram ---------------#        
                cer_subset = CS_MOD06_AUX.cer[current_grid_index][low_water_cloud_selection]
                cer_hist_index = np.searchsorted(self.cer_hist_bound,cer_subset)
                for icer in np.unique(cer_hist_index):
                    self.lwc_cer_hist[ig,icer] += np.sum(cer_hist_index==icer)
                
                #------------ derive cdbc histogram ---------------#        
                cot_subset = CS_MOD06_AUX.tau[current_grid_index][low_water_cloud_selection]
                cdnc_subset = UL.Nd_from_tau_re(cot_subset,cer_subset)
                cdnc_hist_index = np.searchsorted(self.cdnc_hist_bound,cdnc_subset)
                for icdnc in np.unique(cdnc_hist_index):
                    self.lwc_cdnc_hist[ig,icdnc] += np.sum(cdnc_hist_index==icdnc)
                
                #------------ derive cwp-cer joint histogram ---------------#
                for icwp in np.unique(cwp_hist_index):
                    cer_subset_subset = cer_subset[cwp_hist_index==icwp]
                    cer_subset_hist_index = np.searchsorted(self.cer_hist_bound,cer_subset_subset)
                    for icer in np.unique(cer_subset_hist_index):
                        self.lwc_cwp_cer_jhist[ig,icwp,icer]+=np.sum(cer_subset_hist_index==icer)
                
                #------------ derive cwp-cdnc joint histogram ---------------#
                for icwp in np.unique(cwp_hist_index):
                    cdnc_subset_subset = cdnc_subset[cwp_hist_index==icwp]
                    cdnc_subset_hist_index = np.searchsorted(self.cdnc_hist_bound,cdnc_subset_subset)
                    for icdnc in np.unique(cdnc_subset_hist_index):
                        self.lwc_cwp_cdnc_jhist[ig,icwp,icdnc]+=np.sum(cdnc_subset_hist_index==icdnc)
                
                        
                #------------ then process precipitating low water cloud ---------------------------#        
                cwp_subset = CS_MOD06_AUX.cwp[current_grid_index][precip_low_water_cloud_selection]
                cwp_hist_index = np.searchsorted(self.cwp_hist_bound,cwp_subset)
                for icwp in np.unique(cwp_hist_index):
                    self.precip_lwc_cwp_hist[ig,icwp] += np.sum(cwp_hist_index==icwp)
                        
                cer_subset = CS_MOD06_AUX.cer[current_grid_index][precip_low_water_cloud_selection]
                cer_hist_index = np.searchsorted(self.cer_hist_bound,cer_subset)
                for icer in np.unique(cer_hist_index):
                    self.precip_lwc_cer_hist[ig,icer] += np.sum(cer_hist_index==icer)
                    
                #------------ derive cdbc histogram ---------------#        
                cot_subset = CS_MOD06_AUX.tau[current_grid_index][precip_low_water_cloud_selection]
                cdnc_subset = UL.Nd_from_tau_re(cot_subset,cer_subset)
                cdnc_hist_index = np.searchsorted(self.cdnc_hist_bound,cdnc_subset)
                for icdnc in np.unique(cdnc_hist_index):
                    self.precip_lwc_cdnc_hist[ig,icdnc] += np.sum(cdnc_hist_index==icdnc)
                    
                for icwp in np.unique(cwp_hist_index):
                    cer_subset_subset = cer_subset[cwp_hist_index==icwp]
                    cer_subset_hist_index = np.searchsorted(self.cer_hist_bound,cer_subset_subset)
                    for icer in np.unique(cer_subset_hist_index):
                        self.precip_lwc_cwp_cer_jhist[ig,icwp,icer]+=np.sum(cer_subset_hist_index==icer)
                        
                #------------ derive cwp-cdnc joint histogram ---------------#
                for icwp in np.unique(cwp_hist_index):
                    cdnc_subset_subset = cdnc_subset[cwp_hist_index==icwp]
                    cdnc_subset_hist_index = np.searchsorted(self.cdnc_hist_bound,cdnc_subset_subset)
                    for icdnc in np.unique(cdnc_subset_hist_index):
                        self.precip_lwc_cwp_cdnc_jhist[ig,icwp,icdnc]+=np.sum(cdnc_subset_hist_index==icdnc)
                
            
            if verbose:
                ti = time.perf_counter()
                print('processing file', i,' : using time [s]',ti-t1)
        self.total_obs=self.total_obs.reshape(self.nlat,self.nlon)
        self.total_cloud=self.total_cloud.reshape(self.nlat,self.nlon)/self.total_obs
        self.water_cloud=self.water_cloud.reshape(self.nlat,self.nlon)/self.total_obs
        self.low_cloud=self.low_cloud.reshape(self.nlat,self.nlon)/self.total_obs
        self.low_wcf=self.low_wcf.reshape(self.nlat,self.nlon)/self.total_obs
        self.SL_low_wcf=self.SL_low_wcf.reshape(self.nlat,self.nlon)/self.total_obs
        self.low_wcf_precip = self.low_wcf_precip.reshape(self.nlat,self.nlon)/self.low_wcf/self.total_obs
        self.SL_low_wcf_precip = self.SL_low_wcf_precip.reshape(self.nlat,self.nlon)/self.SL_low_wcf/self.total_obs
        self.MODIS_tcf=self.MODIS_tcf.reshape(self.nlat,self.nlon)/self.total_obs
        self.MODIS_tcf_by_ctp = self.MODIS_tcf_by_ctp.reshape(self.nlat,self.nlon)/self.total_obs
        self.MODIS_wcf=self.MODIS_wcf.reshape(self.nlat,self.nlon)/self.total_obs
        self.MODIS_icf=self.MODIS_icf.reshape(self.nlat,self.nlon)/self.total_obs
        self.MODIS_ucf=self.MODIS_ucf.reshape(self.nlat,self.nlon)/self.total_obs
                                             
        self.lwc_cwp_hist = self.lwc_cwp_hist.reshape(self.nlat,self.nlon,self.ncwp)
        self.lwc_cer_hist = self.lwc_cer_hist.reshape(self.nlat,self.nlon,self.ncer)
        self.lwc_cdnc_hist = self.lwc_cdnc_hist.reshape(self.nlat,self.nlon,self.ncdnc)
        self.lwc_cwp_cer_jhist = self.lwc_cwp_cer_jhist.reshape(self.nlat,self.nlon,self.ncwp,self.ncer)
        self.lwc_cwp_cdnc_jhist = self.lwc_cwp_cdnc_jhist.reshape(self.nlat,self.nlon,self.ncwp,self.ncdnc)
        
        self.precip_lwc_cwp_hist = self.precip_lwc_cwp_hist.reshape(self.nlat,self.nlon,self.ncwp)
        self.precip_lwc_cer_hist = self.precip_lwc_cer_hist.reshape(self.nlat,self.nlon,self.ncer)
        self.precip_lwc_cdnc_hist = self.precip_lwc_cdnc_hist.reshape(self.nlat,self.nlon,self.ncdnc)
        self.precip_lwc_cwp_cer_jhist = self.precip_lwc_cwp_cer_jhist.reshape(self.nlat,self.nlon,self.ncwp,self.ncer)
        self.precip_lwc_cwp_cdnc_jhist = self.precip_lwc_cwp_cdnc_jhist.reshape(self.nlat,self.nlon,self.ncwp,self.ncdnc)
        
#         self.lwc_PoP = self.precip_lwc_cwp_cer_jhist / self.lwc_cwp_cer_jhist
        if savefile:
            filename = savepath+'CloudSat_Level3_output'+start_date.strftime("%y%m%d") + '-'+end_date.strftime("%y%m%d")+'.pkl'
            pkl_savefile = open(filename, 'wb')
            pickle.dump(self, pkl_savefile, pickle.HIGHEST_PROTOCOL)
            pkl_savefile.close()

    
    

In [10]:
def process_monthly_mean(start_year,start_month,end_year,end_month):
    import calendar
    from dateutil import relativedelta
    start_date = datetime.date(start_year,start_month,1)
    end_date   = datetime.date(end_year,end_month,calendar.monthrange(end_year,end_month)[1])

    current_month = start_date 
    cl3_array=[]
    while current_month<=end_date:
        sd = current_month
        ed = datetime.date(current_month.year,current_month.month,calendar.monthrange(current_month.year,current_month.month)[1])
        print('processing',sd,ed)
        cl3=CloudSat_Level3(sd,ed,savefile=True,savepath='Level3_data/')
        cl3_array.append(cl3)
        current_month += relativedelta.relativedelta(months=1)
    return cl3_array

In [11]:
# cl3=process_monthly_mean(2013,1,2013,1)

In [81]:
# CDNC_mean = np.zeros([cl3[0].nlat,cl3[0].nlon])
# for ilat in range(cl3[0].nlat):
#     for ilon in range(cl3[0].nlon):
#         CDNC_mean[ilat,ilon]=np.sum(cl3[0].lwc_cdnc_hist[ilat,ilon,0:-1]*cl3[0].cdnc_hist_bound[0:-1])/np.sum(cl3[0].lwc_cdnc_hist[ilat,ilon,0:-1])

# CGM= Cartopy_Global_Contour(cl3[0].lat_grid,cl3[0].lon_grid,CDNC_mean,cmap='rainbow',data_range=[0,400])
# fig,ax=plt.subplots()
# ax.plot(cl3[0].cdnc_hist_bound,np.sum(cl3[0].lwc_cdnc_hist,axis=(0,1)),marker='s')
# ax.plot(cl3[0].cdnc_hist_bound,np.sum(cl3[0].lwc_cwp_cdnc_jhist,axis=(0,1,2)),marker='*',c='r')
# ax.set_xlim(0,500)

# fig,ax=plt.subplots()
# ax.pcolormesh(cl3[0].cwp_hist_bound,cl3[0].cdnc_hist_bound,np.transpose(np.sum(cl3[0].lwc_cwp_cdnc_jhist,axis=(0,1))))
# ax.set_ylim([0,500])
# ax.set_xscale('log')

# fig,ax=plt.subplots()
# PoP = np.sum(cl3[0].precip_lwc_cwp_cdnc_jhist,axis=(0,1))/np.sum(cl3[0].lwc_cwp_cdnc_jhist,axis=(0,1))
# ax.pcolormesh(cl3[0].cwp_hist_bound,cl3[0].cdnc_hist_bound,np.transpose(PoP))
# ax.contour(cl3[0].cwp_hist_bound,cl3[0].cdnc_hist_bound,np.transpose(np.sum(cl3[0].lwc_cwp_cdnc_jhist,axis=(0,1))),color='w')
# ax.set_ylim(0,500)
# ax.set_xscale('log')

# # fig,ax=plt.subplots()
# # ax.plot(cl3[0].cwp_hist_bound,np.sum(cl3[0].lwc_cwp_cer_jhist,axis=(0,1,3)))
# # ax.plot(cl3[0].cdnc_hist_bound,np.sum(cl3[0].lwc_cdnc_hist,axis=(0,1)))
# # ax.set_xlim(0,500)

In [32]:
if __name__ =='__main__':
    import sys
    sy,ey = int(sys.argv[1]),int(sys.argv[2])
    print(sy,ey)
    cl3_list=process_monthly_mean(sy,1,ey,12)

In [ ]:
# CGM= Cartopy_Global_Contour(cl3.lat_grid,cl3.lon_grid,cl3.total_cloud,cmap='rainbow',data_range=[0,100],figure_title='cs_WCF',figure_save='cs_wcf.png')
# CGM= Cartopy_Global_Contour(cl3.lat_grid,cl3.lon_grid,cl3.MODIS_wcf*100.0,cmap='rainbow',data_range=[0,100],figure_title='WCF',figure_save='wcf.png')